# Why Agents?

What is an agent?

An agent is a reference to combining models that can perform some kind of reasoning, like large language models (e.g ChatGPT, Llama2, Mistral, etc...) with tools to give it access to the real world,
so they can do things like browsing the internet, buying stuff, etc...

Ok, so, why is there so much hype around agents right now?

Because Agents are cool! Recently with the advance of LLMs, we've seen them become an amazing tool to do all sorts of things like building apps, browse the internet and more.



SOme neat examples of these kinds of agents can be found in here:

- [AutoGPT](https://github.com/Significant-Gravitas/AutoGPT)
- [GPT-Engineer](https://github.com/gpt-engineer-org/gpt-engineer)
- [BabyAGI](https://github.com/yoheinakajima/babyagi)

Now, today although they seem extremely powerful, agents are still at a very early stage in terms of readiness to deploy as products, something you can atest by listening to Andrej Karpathy talk about agents in this talk here:

- [Karpathy on Agents](https://www.youtube.com/watch?v=fqVLjtvWgq8)

This live-training is all about! Getting you excited about this amazing new technology, understanding it from the ground up but with a focus on practical applications and fun stuff you can do with them! 

# What is an Agent?

An agent is nothing more than some entity that can _think_ and _act_, that's right, in a way you're an agent! 

After all you can think and act on those thoughts like in the case of coming to this live-training:

- Thought: "I want to learn about agents"

- Action: "Go to the internet and research cool platforms where I can learn about agents"

- Thought: "O'Reilly has some awesome courses and live-trainings"

- Action: "Look up O'Reilly courses"

- Thought: "Live-trainings by instructor Lucas are awesome"

- Action: "Schedule live-training about agents with instructor Lucas Soares" (lol)

In a way this is a simplified rendition of what brought you here, obviously not necessarily in this particular order nor these particular sets of thought and action pairs. This particular way of thinking about how to structure thoughts and actions is well represented in the paper: [ReACT](https://arxiv.org/pdf/2210.03629.pdf). 

With regards to LLMs, how can bring this idea to fruition thinking about the LLM model as the reasoning and thinking engine?

We can start simple and just call the openai API to start:

In [ ]:
# !pip install langchain_core
# !pip install langchain
# !pip install openai
# !pip install tiktoken
# !pip install pypdf

In [1]:
import openai
import os
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
from openai import OpenAI
from IPython.display import Markdown
client = OpenAI()

def get_response(prompt_question):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-16k",
        messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                  {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content

output = get_response("Create a simple task list of 3 desktop things I can do on the terminal.")
Markdown(output)

Ok cool, so here we have three ideas of actions to perform:

- Creating directories
- Listing files
- Removing files

Let's transform them into functions that we could call just like in any type of Python-based application.

In [3]:
import subprocess

def create_directory():
    subprocess.run(["mkdir", "test"])

def create_file():
    subprocess.run(["touch", "test.txt"])

def list_files():
    subprocess.run(["ls"])

Notice that here we are not concerned with giving these functions arguments to avoid complicating things, the only thing we should focus on is to have the ability to perform the actions outside the narrow scope of the python script out there in the real-world.

Let's test these functions

In [6]:
!ls

1.0-ls-agents-overview.ipynb
2.0-ls-general-framework-for-building-llm-agents.ipynb
3.0-ls-building-agents-with-langchain.ipynb
4.0-ls-building-agents-langchain-openai-function-calling.ipynb
5.0-ls-use-case1-UNDEFINED.ipynb
assets-resources
function_calls_chatgpt.py
langchain_agents


Ok, we start with this folder containing a few notebooks, scripts and sub-directories.

Let's test each function and inspect its output behavior:

In [7]:
create_directory()

!ls

1.0-ls-agents-overview.ipynb
2.0-ls-general-framework-for-building-llm-agents.ipynb
3.0-ls-building-agents-with-langchain.ipynb
4.0-ls-building-agents-langchain-openai-function-calling.ipynb
5.0-ls-use-case1-UNDEFINED.ipynb
assets-resources
function_calls_chatgpt.py
langchain_agents
test


In [8]:
create_file()

!ls

1.0-ls-agents-overview.ipynb
2.0-ls-general-framework-for-building-llm-agents.ipynb
3.0-ls-building-agents-with-langchain.ipynb
4.0-ls-building-agents-langchain-openai-function-calling.ipynb
5.0-ls-use-case1-UNDEFINED.ipynb
assets-resources
function_calls_chatgpt.py
langchain_agents
test
test.txt


In [17]:
list_files()

1.0-ls-agents-overview.ipynb
2.0-ls-general-framework-for-building-llm-agents.ipynb
3.0-ls-building-agents-with-langchain.ipynb
4.0-ls-building-agents-langchain-openai-function-calling.ipynb
5.0-ls-use-case1-UNDEFINED.ipynb
assets-resources
function_calls_chatgpt.py
langchain_agents
test


Ok great all the functions are working!

Now, let's imagine that we wanted to create an agent that would perform these actions for us based on some input that we give it, how can we connect models that we know and can use today like ChatGPT, with these tools that do stuff in the real world?

To answer this question, how about we give a task to the model, and for that task we ask it to list the steps that it needs to perform to complete the task, and then for each of those steps we would ask the model to decide whether or not a function should be called to execute that task? 

This is what the now famous paper ['Toolformer'](https://arxiv.org/pdf/2302.04761.pdf) demonstrated!

They showed that today's advanced LLMs like the gpt-series could teacha themselves how to properly call and use external tools!

Isn't that awesome???

So, let's see if we can hack our way into connecting the llm response with the functions that we want that llm to use.

In [4]:
def get_response(prompt_question):
    response = client.chat.completions.create(
        model=self.model,
        messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content

def create_directory(directory_name):
    subprocess.run(["mkdir", directory_name])

def create_file(file_name):
    subprocess.run(["touch", file_name])

def list_files():
    subprocess.run(["ls"])

OK, cool! now Notice that, here we added single parameters to the functions: `create_directory(), create_file()`, and we did this so
that we can actually do real things instead of just always creating the same folders over and over.

Now, how can we actually put it all together so that given a task, a model can:

- Plan the task
- Execute actions to complete the task
- Know when to call a function

????

This is actually an interesting problem, let's understand why is that the case by trying to hack our way into putting all of these together:

In [6]:
def get_response(prompt_question, model="gpt-3.5-turbo-16k"):
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content

def create_directory(directory_name):
    subprocess.run(["mkdir", directory_name])

def create_file(file_name):
    subprocess.run(["touch", file_name])

def list_files():
    subprocess.run(["ls"])

    

task_description = "Create a folder called 'lucas-the-agent-master'. Inside that folder, create a file called 'the-10-master-rules.md"
output = get_response(f"""Given this task: {task_description}, \n
                            Consider you have access to the following functions:
                            
    def create_directory(directory_name):
        '''Function that creates a directory given a directory name.'''
        subprocess.run(["mkdir", directory_name])
    
    def create_file(file_name):
        '''Function that creates a file given a file name.'''
        subprocess.run(["touch", file_name])
    
    def list_files():
       '''Function that lists all files in the current directory.'''
        subprocess.run(["ls"])
    
    Your output should be the first function to be executed to complete the task containing the necessary arguments.
    The OUTPUT SHOULD ONLY BE THE PYTHON FUNCTION CALL and NOTHING ELSE.
    """)

Markdown(output)

create_directory('lucas-the-agent-master')

Hey! Look at that the output is that function! Now, all we need is find a way to execute this function. We can use Python's built in `exec` method for that:

In [27]:
exec("model." + output)

In [29]:
!ls -d */ | grep lucas

lucas-the-agent-master/


Yessss! We did it! All we had to do is to use the Python builtin method `exec` connected with the function call we got from the model's response!

This is great, but what if we wanted to perform multiple actions?

How about changing our prompt so that our output is a python list of function calls which we can later programmatically call?

Let's try that:

In [8]:
task_description = "Create a folder called 'lucas-the-agent-master'. Inside that folder create a file called 'the-10-master-rules.md'."
output = get_response(f"""Given a task that will be fed as input, and consider you have access to the following functions:
                            
    def create_directory(directory_name):
        '''Function that creates a directory given a directory name.'''
        subprocess.run(["mkdir", directory_name])
    
    def create_file(file_name):
        '''Function that creates a file given a file name.'''
        subprocess.run(["touch", file_name])
    
    def list_files():
       '''Function that lists all files in the current directory.'''
        subprocess.run(["ls"])  
    .
    Your output should be the a list of function calls to be executed to complete the task containing the necessary arguments.
    For example:
    
    task: 'create a folder named test-dir'
    output_list: [create_directory('test-dir')]
    
    task: 'create a file named file.txt'
    output_list: [create_file('file.txt')]
    
    task: 'Create a folder named lucas-dir and inside that folder create a file named lucas-file.txt'
    output_list: [create_directory('lucas-dir'), create_file('lucas-dir/lucas-file.txt')]
    
    The OUTPUT SHOULD ONLY BE A PYTHON LIST WITH THE FUNCTION CALLS INSIDE and NOTHING ELSE.
    task: {task_description}
    output_list:\n
    """)

Markdown(output)

[create_directory('lucas-the-agent-master'), create_file('lucas-the-agent-master/the-10-master-rules.md')]

In [ ]:
exec(output)

In [64]:
!ls lucas-the-agent-master/

the-10-master-rules.md


At this point we can start identifying a lot of issues with this approach despite our early sucess:

- Uncertainty of model's outputs can affect our ability to reliably call the functions
- We need more structured ways to prepare the inputs of the function calls
- We need better ways to put everything together (just feeding the entire functions like this makes it a very clunky and non-scalable framework for more complex cases)

There are many more issues but starting with these, we can now look at frameworks and see how they fix these issues and with that in mind understand what is behind their implementations!

I personally think this is a much better way to understand what is going on behind agents in practice rather than just use the more higher level frameworks right of the bat!

# OpenAI Functions

Ok, let's first understand how [OpenAI](https://openai.com/) the company behind ChatGPT, allows for these function call implementations in its API.

OpenAI implemented a [function calling API](https://platform.openai.com/docs/guides/function-calling) which is a standard way to connect their models to outside tools like in the very simple example we did above.

According to their [official documentation](https://platform.openai.com/docs/guides/function-calling#:~:text=The%20basic%20sequence,to%20the%20user.) the sequence of steps for function calling is as follows:
1. Call the model with the user query and a set of functions defined in the functions parameter.
2. The model can choose to call one or more functions; if so, the content will be a stringified JSON object adhering to your custom schema (note: the model may hallucinate parameters).
3. Parse the string into JSON in your code, and call your function with the provided arguments if they exist.
4. Call the model again by appending the function response as a new message, and let the model summarize the results back to the user.

Below is an example taken from their official documentation:

In [66]:
from openai import OpenAI
import json

client = OpenAI()

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
output = run_conversation()
output

ChatCompletion(id='chatcmpl-8XuJYsA426IZfzU8rSbjm7jNKhHwL', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The current weather in San Francisco is 72°C, in Tokyo it's 10°C, and in Paris it's 22°C.", role='assistant', function_call=None, tool_calls=None))], created=1703091988, model='gpt-3.5-turbo-1106', object='chat.completion', system_fingerprint='fp_772e8125bb', usage=CompletionUsage(completion_tokens=28, prompt_tokens=171, total_tokens=199))

In [68]:
output.choices[0].message.content

"The current weather in San Francisco is 72°C, in Tokyo it's 10°C, and in Paris it's 22°C."

Ok neat! So what's happening here is that they implemented the useful json schema to make the calls more reliable by structuring the input output flows, and trained the model on this schema to be better at calling functions.

Let's look at how our previous model with those three simple functions: `create_directory()`, `create_file()`, and `list_files()` would be implemented using OpenAI's function calling approach:

In [80]:
import json

def create_directory(directory_name):
    """Function that creates a directory given a directory name."""""
    subprocess.run(["mkdir", directory_name])
    return json.dumps({"directory_name": directory_name})


tool_create_directory = {
    "type": "function",
    "function": {
        "name": "create_directory",
        "description": "Create a directory given a directory name.",
        "parameters": {
            "type": "object",
            "properties": {
                "directory_name": {
                    "type": "string",
                    "description": "The name of the directory to create.",
                }
            },
            "required": ["directory_name"],
        },
    },
}

tools = [tool_create_directory]    

In [82]:
import json

def run_terminal_task():
    messages = [{"role": "user", "content": "Create a folder called 'lucas-the-agent-master'."}]
    tools = [tool_create_directory]  
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-16k",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "create_directory": create_directory,
        }
        messages.append(response_message)
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                directory_name=function_args.get("directory_name"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
        second_response = client.chat.completions.create(
            model="gpt-3.5-turbo-16k",
            messages=messages,
        )
        return second_response

output = run_terminal_task()
output

ChatCompletion(id='chatcmpl-8XuXUzjPlY3vMkexcmsbq5IuKWmOT', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="The folder 'lucas-the-agent-master' has been created successfully.", role='assistant', function_call=None, tool_calls=None))], created=1703092852, model='gpt-3.5-turbo-16k-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=14, prompt_tokens=57, total_tokens=71))

In [83]:
output.choices[0].message.content

"The folder 'lucas-the-agent-master' has been created successfully."

In [84]:
!ls 

1.0-ls-agents-overview.ipynb
2.0-ls-general-framework-for-building-llm-agents.ipynb
3.0-ls-building-agents-with-langchain.ipynb
4.0-ls-building-agents-langchain-openai-function-calling.ipynb
5.0-ls-use-case1-UNDEFINED.ipynb
assets-resources
function_calls_chatgpt.py
langchain_agents
lucas-the-agent-master
test


Great! We implemented openai function calling for creating directories! We could evolve this approach but let's stop for now.

See more info on these examples from OpenAI's [official cookbook](https://cookbook.openai.com/examples/how_to_call_functions_with_chat_models).

# Agents

Ok, this is awesome, we can now use openai functions connected to the ChatGPT API endpoint to allow it to perform actions in the real world, and the calling of said functions is made easier by leveraging the json schema which structures how the model would call the function. 

This is a great setup and tool but more complex tasks, we want to have more control over the process these potential agents will go through in order to make them as reliable as possible. We want to control things like:

- What goes in and out of prompts
- What goes in and out of each thought and action pair stage the agent goes through when doing something in the real-world
- A convenient interface to compose agents with useful building blocks (also leverage open source LLMs if we want to)

I could go on but this is enough for now.

For scenarios like these where just connecting a model to some tools won't cut it, a really great framework that has had a lot of sucess recently is [LangChain](https://python.langchain.com/docs/get_started/introduction).

LangChain is a framework that allows for building LLM-powered applications by giging developers the ability to build and compose building blocks like prompts, models, tools and so on to develop complex and interesting applications.

To get an overview of the capabilities of LangChain, let's take a look below at a LangChain implementation of our simple agent that can create directories:

In [91]:
from langchain.tools import tool
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentExecutor
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

@tool
def create_directory(directory_name):
    """Function that creates a directory given a directory name."""""
    subprocess.run(["mkdir", directory_name])
    return json.dumps({"directory_name": directory_name})


tools = [create_directory]

llm_chat = ChatOpenAI(temperature=0)

prompt = ChatPromptTemplate.from_messages(
[
    ("system","You are very powerful assistant that helps\
                users perform tasks in the terminal."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

llm_with_tools = llm_chat.bind(functions=[format_tool_to_openai_function(t) for t in tools])

agent = (
{
    "input": lambda x: x["input"],
    "agent_scratchpad": lambda x: format_to_openai_function_messages(
        x["intermediate_steps"]
    ),
}
| prompt
| llm_with_tools
| OpenAIFunctionsAgentOutputParser())

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

action_input = "Create a folder called 'lucas-the-agent-master'."

agent_executor.invoke({"input": action_input})



> Entering new AgentExecutor chain...

Invoking: `create_directory` with `{'directory_name': 'lucas-the-agent-master'}`


{"directory_name": "lucas-the-agent-master"}Folder 'lucas-the-agent-master' has been created successfully.

> Finished chain.


{'input': "Create a folder called 'lucas-the-agent-master'.",
 'output': "Folder 'lucas-the-agent-master' has been created successfully."}

Ok great! Now, let's walkthrough what is going on here!

Define the custom tool function:
```python
@tool
def create_directory(directory_name):
    """Function that creates a directory given a directory name."""
    subprocess.run(["mkdir", directory_name])
    return json.dumps({"directory_name": directory_name})
```

- Create a list of tools:

```python
tools = [create_directory]
```

- Initialize the chat model:

```python
llm_chat = ChatOpenAI(temperature=0)
```

- Create a chat prompt template:

```python
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are very powerful assistant that helps users perform tasks in the terminal."),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
```

- Bind the chat model with the tools:

```python
llm_with_tools = llm_chat.bind(functions=[format_tool_to_openai_function(t) for t in tools])
```

- Define the agent:

```python
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(x["intermediate_steps"]),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)
```

- Create an instance of AgentExecutor:

```python
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
```

- Invoke the agent with an input:

```python
action_input = "Create a folder called 'lucas-the-agent-master'."
agent_executor.invoke({"input": action_input})
```

In the given setup, the intermediate_steps and agent_scratchpad are used to represent the history of the agent's thought process and the intermediate results generated during that process. Here's a breakdown of each component:

• intermediate_steps: This is a sequence of tuples that contains the agent's actions and the corresponding tool outputs at each step of the conversation. It provides a record of the agent's decision-making process and the information it has gathered along the way.

• agent_scratchpad: This is a placeholder variable that holds the intermediate_steps in the prompt template. It allows the agent to access and utilize the historical information during its decision-making process.

• MessagesPlaceholder: This is a component in the prompt template that acts as a placeholder for the agent_scratchpad. It specifies where the agent_scratchpad should be inserted in the conversation. In the given setup, the MessagesPlaceholder is placed after the user input, indicating that the agent_scratchpad should be included in the conversation after the user's message.


By including the intermediate_steps and agent_scratchpad in the prompt template and passing them to the agent, the setup allows the agent to have access to its previous actions and tool outputs, enabling it to make informed decisions and generate more contextually relevant responses.

The essence of this script is to define a custom tool function create_directory that creates a directory given a directory name. It initializes an agent with the tool function and a chat model. The agent is then invoked with an input to execute the tool function. The output of the agent is printed.

Don't worry about understanding the syntax, let's just discuss at a higher level what components are at play here:

- `@tool` decorator: transforms a regular function into a usable tool for a LangChain-based agent. 
- `ChatOpenAI` - LangChain's implementation of the OpenAI API for chat models (like `gpt-3.5-turbo-1106`).
- `ChatPromptTemplate`: something that allows you to programatically abstract over the chat prompt for the chat model.
- `format_tool_to_openai_function`: LangChain's method to convert a tool to one formatted according to OpenAI's scheme.
- `format_to_openai_function_message`: LangChain's method to format a message to the OpenAI function calling format.
- `OpenAIFunctionsAgentOutputParser`: LangChain's output parsing for outputs of an OpenAI function.
- `agent = ... | ... | ... |`: Langchain's Expression Language interface which allows you to build these complex and cool agents by giving you a simple interface that uses the Unix pipe symbol `|` to compose building blocks like models and prompts.
- `AgentExecutor`: LangChain framework that serves as the runtime for an agent. It is responsible for executing the agent's decision-making process, invoking tools, and generating responses.

When we use an agent we want that agent to have access to its previous outputs and decision-making process in order for it to make more informed decisions. Therefore, LangChain allows you to set all that up yourself so you have ultimate control over what is going on.

On the next notebook we'll dive into building Agents like these leveraging the LangChain framework. 

# References

- [HuggingGPT](https://github.com/microsoft/JARVIS)
- [Gen Agents](https://arxiv.org/pdf/2304.03442.pdf)
- [WebGPT](https://www.semanticscholar.org/paper/WebGPT%3A-Browser-assisted-question-answering-with-Nakano-Hilton/2f3efe44083af91cef562c1a3451eee2f8601d22)
- [LangChain](https://python.langchain.com/docs/get_started/introduction)
- [OpenAI](https://openai.com/)
- [OpenAI Function Calling](https://platform.openai.com/docs/guides/function-calling)
- [AutoGPT](https://github.com/Significant-Gravitas/AutoGPT)
- [GPT-Engineer](https://github.com/gpt-engineer-org/gpt-engineer)
- [BabyAGI](https://github.com/yoheinakajima/babyagi)
- [Karpathy on Agents](https://www.youtube.com/watch?v=fqVLjtvWgq8)
- [ReACT Paper](https://arxiv.org/abs/2210.03629)